In [7]:
pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from dotenv import load_dotenv


In [9]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [12]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [13]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [14]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [15]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [16]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or feelings, but I can certainly highlight some popular data tools that are widely appreciated in the industry for their capabilities. Tools like Collibra, Informatica, and Alation are often favored for their robust data governance features, including data cataloging, data lineage, and metadata management. These tools help organizations ensure data quality, compliance, and effective data management. The choice of tool often depends on the specific needs and existing infrastructure of an organization.


# Load storage contexts

## Load vector storage context

In [17]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [18]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [19]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [20]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x17161d270>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x17fc452a0>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x17161e350>, property_graph_store=None)


# Load index

## Load vector index

In [21]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [22]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [23]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs personnes et équipes clés :

1. **Mme Laurent** : Elle a exprimé le besoin d'accéder à des données plus complètes et en temps réel. Vous pouvez l'impliquer pour définir les besoins en données et les indicateurs clés de performance (KPIs) à suivre.

2. **M. Dupont** : En tant que Directeur de production, il peut être impliqué pour s'assurer que les données sont complètes et fiables. Il peut également aider à identifier les champs critiques dans le Datawarehouse et à mettre en place des contrôles automatiques pour garantir la qualité des données.

3. **Équipes IT** : Elles seront essentielles pour la mise en place des processus ETL, la validation des données, et la gestion de l'infrastructure technique nécessaire pour le projet.

4. **Équipes Opérationnelles** : Elles doivent être formées pour comprendre l'importance de la saisie complète des données et pour collaborer avec les équipes IT afin de définir les priorités des données critiques.

5. **Responsable de la Base Opération** : Cette personne peut aider à fournir une documentation claire et détaillée de la base de données "Opérations" pour garantir une compréhension commune et faciliter l'utilisation et la maintenance de la base.

Ensemble, ces parties prenantes peuvent collaborer pour assurer le succès du projet en garantissant que les données opérationnelles sont bien gérées, complètes, et utilisées efficacement pour atteindre vos objectifs.</b>

## Load graph index

In [25]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [26]:
nx_graph = graph_index.get_networkx_graph()

In [27]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 87


In [28]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 116 nodes and 87 edges
Nodes: ['Document', 'Aspects techniques du datawarehouse', 'Spécifications des flux de données', 'Caractéristiques techniques', 'Description', 'Datawarehouse', 'Réservoir central', 'Données consolidées', 'Élément crucial', 'Fiabilité', 'Postgresql 14', 'Base de données', 'Modèle en étoile', 'Structure', 'Directeur de la production', 'Accès', 'Politiques strictes', 'Administrateurs système', 'Indirect', 'Contrats', 'Erp', 'Système central de gestion', 'Données relatives à', 'Visibilité en temps réel', 'Interfaces', 'Transfert des données', 'Informations précises et à jour', 'Interface', 'Etl', 'Outil', 'Talend data integration', 'Source', 'Erp (sap', 'Destination', 'Tables de faits et dimensions du datawarehouse', 'Planification', 'Ordonnanceur cron', 'Alertes', "Emails et logs en cas d'échec", 'Gestion', 'Données', 'Fragmentées', 'Validation', 'Contrôles sur les types et les règles métier', 'Reporting', 'Tableau de bord sur les échecs de flux', 'Rétent

### Set retriever

In [29]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [30]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
INFO:llama_index.core.indices.knowledge_graph.

In [31]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes et définir des rôles spécifiques :

1. **Équipes Opérationnelles** : Organisez une formation pour ces équipes afin de les préparer à gérer et analyser les données opérationnelles. Elles joueront un rôle clé dans la collecte et la validation des données.

2. **Équipe IT/Technique** : Impliquez-les pour gérer les aspects techniques du Datawarehouse, y compris les flux de données et les transformations nécessaires. Ils seront responsables de la mise en place des règles de validation automatique dans les processus ETL.

3. **Responsables de Projet** : Assurez-vous d'avoir un responsable de projet qui peut coordonner les différentes équipes et s'assurer que le projet respecte les délais et les objectifs fixés.

4. **Analystes de Données** : Ils seront essentiels pour l'analyse et l'interprétation des données, permettant de tirer des insights stratégiques pour l'entreprise.

5. **Gestionnaires de l'ERP** : Puisque l'ERP alimente le Datawarehouse, il est crucial d'impliquer ceux qui gèrent l'ERP pour garantir la synchronisation et la fiabilité des données.

Ensemble, ces parties prenantes peuvent collaborer pour assurer le succès de votre projet data en utilisant efficacement les données opérationnelles.</b>

## Load onto index

In [32]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.


In [33]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [34]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [35]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [36]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes et définir des rôles spécifiques :

1. **Équipes Opérationnelles et IT** : Collaborez étroitement avec ces équipes pour définir les priorités des données critiques et assurer une intégration fluide des données dans le Datawarehouse.

2. **Responsable de Projet** : Une personne comme Jean Dupont, qui a déjà une expérience dans la gestion des opérations, pourrait être responsable de superviser le projet et de coordonner les différentes équipes.

3. **Équipe de Formation** : Organisez une session de formation pour les équipes terrain afin de les sensibiliser à l'importance de la saisie complète des données dans l'ERP, ce qui est crucial pour la qualité des données opérationnelles.

4. **Spécialistes en ETL** : Mettez en place des contrôles automatiques pour détecter et signaler les champs manquants lors des processus ETL, garantissant ainsi la fiabilité des données transférées.

5. **Développeurs** : Développez un accès personnalisé avec une interface d'utilisation pour faciliter l'interaction avec les données opérationnelles.

Ces rôles et actions contribueront à la réussite de votre projet data en assurant une gestion efficace des données opérationnelles.</b>

# Visualize knowledge graph

In [37]:
nx_graph = graph_index.get_networkx_graph()

In [38]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 87


In [39]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 116 nodes and 87 edges
Nodes: ['Document', 'Aspects techniques du datawarehouse', 'Spécifications des flux de données', 'Caractéristiques techniques', 'Description', 'Datawarehouse', 'Réservoir central', 'Données consolidées', 'Élément crucial', 'Fiabilité', 'Postgresql 14', 'Base de données', 'Modèle en étoile', 'Structure', 'Directeur de la production', 'Accès', 'Politiques strictes', 'Administrateurs système', 'Indirect', 'Contrats', 'Erp', 'Système central de gestion', 'Données relatives à', 'Visibilité en temps réel', 'Interfaces', 'Transfert des données', 'Informations précises et à jour', 'Interface', 'Etl', 'Outil', 'Talend data integration', 'Source', 'Erp (sap', 'Destination', 'Tables de faits et dimensions du datawarehouse', 'Planification', 'Ordonnanceur cron', 'Alertes', "Emails et logs en cas d'échec", 'Gestion', 'Données', 'Fragmentées', 'Validation', 'Contrôles sur les types et les règles métier', 'Reporting', 'Tableau de bord sur les échecs de flux', 'Rétent

# (Simple) Query the vectors

In [40]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [41]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes clés :

1. **Mme Laurent** : Elle peut être impliquée pour s'assurer que les données opérationnelles sont complètes et en temps réel, ce qui est crucial pour optimiser les performances et suivre les KPIs locaux. Elle peut aider à identifier les besoins spécifiques en données pour votre projet.

2. **M. Dupont** : En tant que directeur de production, il peut jouer un rôle essentiel dans la définition des champs critiques et prioritaires à inclure dans le Datawarehouse. Il peut également aider à mettre en place des contrôles automatiques pour garantir la complétude des données et collaborer avec les équipes IT pour définir les priorités des données critiques.

3. **Équipes IT** : Elles seront nécessaires pour la mise en place des processus ETL, la validation des données, et pour assurer la sécurité et la sauvegarde de la base de données. Elles peuvent également organiser des formations pour les équipes opérationnelles.

4. **Équipes Opérationnelles** : Elles doivent être formées sur l'importance de la saisie complète des données dans l'ERP et peuvent fournir des retours sur les besoins opérationnels spécifiques.

Ensemble, ces parties prenantes peuvent collaborer pour garantir que votre projet data est bien structuré, sécurisé, et répond aux besoins opérationnels.</b>

# (Simple) Query the knowledge graph 

In [42]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
INFO:llama_index.core.indices.knowledge_graph.

In [43]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes et définir des rôles spécifiques :

1. **Équipes Opérationnelles** : Organisez une formation pour ces équipes afin de les préparer à gérer et analyser les données opérationnelles. Elles joueront un rôle clé dans la collecte et la validation des données.

2. **Équipe IT/Technique** : Impliquez-les pour gérer les aspects techniques du Datawarehouse, y compris les flux de données et les transformations nécessaires. Ils seront responsables de la mise en place des règles de validation automatique dans les processus ETL.

3. **Responsables de Projet** : Assurez-vous d'avoir un responsable de projet qui peut coordonner les différentes équipes et s'assurer que les objectifs du projet sont atteints. Ils peuvent également gérer les relations entre les opérations et les projets spécifiques.

4. **Experts en ERP** : Puisque l'ERP alimente le Datawarehouse, il est crucial d'avoir des experts qui comprennent les modules clés et les flux d'intégration pour garantir la continuité et la fiabilité des échanges de données.

Ensemble, ces parties prenantes peuvent collaborer pour assurer le succès du projet en exploitant efficacement les données opérationnelles.</b>

# (Simple) Query the onto graph 

In [44]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [45]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pourriez impliquer Mme Sophie Laurent et M. Philippe Martin, qui ont approuvé le compte rendu. Vous pourriez également envisager de réaliser un audit des données disponibles pour identifier les champs pertinents, développer un accès personnalisé avec une interface d'utilisation, et organiser une session de formation pour la prise en main des nouveaux outils.</b>

## (Node retriever)

In [46]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?")


for node in nodes:
    print(node.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Here are some facts extracted from the provided text:

Base Opération -> IMPROVES -> pilotage des activités opérationnelles

Prochaines Étapes

1. Audit des données disponibles dans la Base Opération et identification des champs pertinents pour Mme Laurent (15 janvier 2025).
2. Développement d’un accès personnalisé avec interface d’utilisation (31 janvier 2025).
3. Organisation d’une session de formation pour la prise en main des nouveaux outils (15 février 2025).

---

**Validation** : Le compte rendu a été approuvé par Mme Sophie Laurent et M. Philippe Martin.  
**Rédigé par** : [Nom du Rédacteur]
Here are some facts extracted from the provided text:

date_creation -> HAS_DESCRIPTION -> Date et heure de création de l'opération

Création 

# Have a real chat with your data

## Set up the engines

### Vector engines

In [47]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu es un consultant spécialisé en stratégie et gouvernance des données."
        "Pour des idées de projets, tu vas proposer une stratégie et une gouvernance pour la mettre en oeuvre"
        "Tu feras attention à mettre en avant le Return On Investment (ROI) du projet proposé"
    ),
    verbose=False,
)

### Graph engines

In [48]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [49]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

## Generate

In [60]:
graph_chat_engine.reset()

In [61]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu es un consultant spécialisé en stratégie et gouvernance des données."
        "Tu cites en priorité les métiers, les bases de données ou les champs de données présents dans ta base de connaissance."
        "Pour des idées de projets, tu vas proposer une stratégie et une gouvernance pour la mettre en oeuvre"
        "Tu feras attention à mettre en avant le Return On Investment (ROI) du projet proposé"
    ),
    verbose=False,
)

In [62]:
response_stream = graph_chat_engine.stream_chat("""D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?""")

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?
Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?
Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 6ffd556c-718a-46fb-a6f6-6e30001b1a62: ---
entite: base
parent: "[[ERP]]"
---
La base de données "Opérations" est un...
> Querying with idx: 6ffd556c-718a-46fb-a6f6-6e30001b1a62: ---
entite: base
parent: "[[ERP]]"
---
La base de données "Opérations" est un...
> Querying with idx: 6ffd556c-718a-46fb-a6f6-6e30001b1a62: ---
entite: base
parent: "[[ERP]]"
---
La base de données "Opérations" est un...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: b51fda10-72af-4ac6-bab9-edd7267fd758: Relations avec d'autres Tables

- **Table `utilisateurs`** :
    - Relation p...
> Querying with idx: b51fda10-72af-4ac6-bab9-edd7267fd758: Relations avec d'autres Tables

- **Table `utilisateurs`** :
    - Relation p...
> Querying with idx: b51fda10-72af-4ac6-bab9-edd7267fd758: Relations avec d'autres Tables

- **Table `utilisateurs`** :
    - Relation p...
INFO:llama_index.core.indices.knowledge_graph.

In [63]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Lancer un projet data avec vos données opérationnelles est une initiative stratégique qui peut apporter une valeur significative à votre organisation. Voici une stratégie et une gouvernance pour mettre en œuvre ce projet, en mettant l'accent sur le ROI :

### 1. **Constitution de l'équipe projet**

#### a. **Responsable de projet (Project Manager)**
   - **Rôle** : Superviser l'ensemble du projet, coordonner les équipes, gérer le budget et le calendrier.
   - **ROI** : Assure que le projet est livré à temps et dans les limites du budget, maximisant ainsi l'efficacité des ressources.

#### b. **Data Scientist/Analyst**
   - **Rôle** : Analyser les données opérationnelles pour extraire des insights exploitables.
   - **ROI** : Identifie les 

In [64]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [65]:
section_modele = """ 


"""

In [66]:
response_stream = graph_chat_engine.stream_chat("""D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible, quel serait le schéma de données idéal pour ce projet. Utilise le schéma de données standard Frictionless pour le décrire. Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: D'après tes connaissances et ta vision du monde, quel serait le schéma de données idéal pour un projet data utilisant des données opérationnelles, en utilisant le schéma de données standard Frictionless pour le décrire ? Peux-tu imiter le style d'écriture de la {section_modele} ?
Condensed question: D'après tes connaissances et ta vision du monde, quel serait le schéma de données idéal pour un projet data utilisant des données opérationnelles, en utilisant le schéma de données standard Frictionless pour le décrire ? Peux-tu imiter le style d'écriture de la {section_modele} ?
Condensed question: D'après tes connaissances et ta vision du monde, quel serait le schéma 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 0ef83e4e-956b-4a9a-a07c-c98c95636658: ---
entite: base
---
Ce document a pour objectif de décrire les aspects techn...
> Querying with idx: 0ef83e4e-956b-4a9a-a07c-c98c95636658: ---
entite: base
---
Ce document a pour objectif de décrire les aspects techn...
> Querying with idx: 0ef83e4e-956b-4a9a-a07c-c98c95636658: ---
entite: base
---
Ce document a pour objectif de décrire les aspects techn...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: b8f1fe18-2d23-4fc9-ae6c-0a9ccabf3cdd: Schéma de la Base de Données
> Querying with idx: b8f1fe18-2d23-4fc9-ae6c-0a9ccabf3cdd: Schéma de la Base de Données
> Querying with idx: b8f1fe18-2d23-4fc9-ae6c-0a9ccabf3cdd: Schéma de la Base de Données
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Que

In [67]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Pour concevoir un schéma de données idéal pour votre projet en utilisant le standard Frictionless, il est essentiel de structurer vos données de manière à maximiser leur utilité et leur interopérabilité. Voici un exemple de schéma de données en utilisant le format Frictionless, qui est un standard ouvert pour la publication de données tabulaires :

### {section_modele}

#### Nom du schéma : **OpérationsDataSchema**

#### Description : 
Ce schéma de données est conçu pour structurer les données opérationnelles de l'organisation, facilitant ainsi l'analyse et l'extraction d'insights pour améliorer l'efficacité et la prise de décision.

#### Ressources :

1. **Table : Transactions**

   - **Description** : Contient les enregistrements de tout

In [68]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [69]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandgovern/output116n87e.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 